Welcome to YOLODarknet google colab notebook.

IMPORTANT NOTE: Before doing anything, enable GPU in 
Runtime -> Change runtime type -> select GPU.

In [0]:
### Run this cell to setup everything and compile functions ###

# This cell imports the drive library and mounts your Google Drive as a VM local drive. You can access to your Drive files 
# using this path "/content/gdrive/My Drive/"
from google.colab import drive
drive.mount('/content/gdrive')

# Necessary tree package
!sudo apt-get install tree
!tree /content/gdrive/My\ Drive/darknet/

# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!tar -xzvf gdrive/My\ Drive/darknet/cuDNN/cudnn-10.1-linux-x64-v7.6.5.32.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h
# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

# Clone darknet to environment
!git clone https://github.com/kriyeng/darknet/
%cd darknet
# Check the folder
!ls
# I have a branch where I have done the changes commented above
!git checkout feature/google-colab
#Compile Darknet
!make
#Copies the Darknet compiled version to Google drive
!cp ./darknet /content/gdrive/My\ Drive/darknet/bin/darknet

# Define a function to show an image with a certain path
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)

In [0]:
### Run this cell to train custom models ###
# Must previously have moved necessary train files to google drive folders acording to the following execute commands

# Train custom models
!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/yolov3.cfg" "/content/gdrive/My Drive/darknet/darknet53.conv.74" -dont_show 

# Start training at the point where the last runtime finished
!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/yolov3.cfg" "/content/gdrive/My Drive/darknet/weights/yolov_last.weights" -dont_show 

In [0]:
### Run this cell to test models ###
# Leave either IMAGE module uncommented or VIDEO module uncommented, dont leave both uncommented.

# Move necessary detector files to darknet/data folder. Comment this three lines after first execution
!cp "/content/gdrive/My Drive/darknet/backup/yolov3_last.weights" data/
!cp "/content/gdrive/My Drive/darknet/obj.data" data/
!cp "/content/gdrive/My Drive/darknet/yolov3.cfg" data/

# Test model (IMAGE)
# Move desired test image to darknet/data folder
!cp "/content/gdrive/My Drive/darknet/test/testimage4.jpg" data/
# Command to run the detector
!./darknet detector test data/obj.data data/yolov3.cfg data/yolov3_last.weights data/testimage4.jpg -dont_show
# Move result image to darknet/results google drive folder
!cp predictions.jpg "/content/gdrive/My Drive/darknet/results/"
# Show result image
imShow('predictions.jpg')

# Test model (VIDEO)
# Move desired test video to darknet/data folder
#!cp "/content/gdrive/My Drive/darknet/test/testvideo.mp4" data/
# Command to run the detector
#!./darknet detector demo data/obj.data data/yolov3.cfg data/yolov3_last.weights -dont_show data/testvideo.mp4 -i 0 -out_filename result.avi
# Move result video to darknet/results google drive folder
#!cp result.avi "/content/gdrive/My Drive/darknet/results/"

In [0]:
### Just random usefull commands ###

# Use this command to stop "Run all" routine on this command's line
assert False

# Get yolov3 weights
!wget https://pjreddie.com/media/files/yolov3.weights

# CUDA: Let's check that Nvidia CUDA is already pre-installed and which version is it. In some time from now maybe you 
!/usr/local/cuda/bin/nvcc --version

# Symbolic link to google drive paths, replaces a huge path with a certain defined path
!ln -s /content/gdrive/My\ Drive/darknet/ /mydrive
!ls /mydrive